# Part 1: Launch a Duet Server and upload data

In [1]:
import syft as sy
duet = sy.launch_duet()

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!
♫♫♫ > Duet Server ID: 0355643b0357fbf356acabe406031185

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.duet("0355643b0357fbf356acabe406031185")

♫♫♫ > STEP 2: Ask your partner for their Client ID and enter it below!

♫♫♫ > Connecting...


C:\Users\dylan\anaconda3\envs\new_pysyft\lib\site-packages\aiortc\rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
C:\Users\dylan\anaconda3\envs\new_pysyft\lib\site-packages\aiortc\rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore


In [2]:
import torch as th
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd

Read in data

In [3]:
df_X_train_1 = pd.read_csv('ETL/X_train_1.csv')
df_X_val_1 = pd.read_csv('ETL/X_val_1.csv')
df_y_train_1 = pd.read_csv('ETL/y_train_1.csv')
df_y_val_1 = pd.read_csv('ETL/y_val_1.csv')

Conver to NumPy array

In [4]:
X_train_1_np = df_X_train_1.to_numpy()
X_val_1_np = df_X_val_1.to_numpy()
y_train_1_np = df_y_train_1.to_numpy()
y_val_1_np = df_y_val_1.to_numpy()

Convert to Torch tensor

In [5]:
X_train_1 = th.from_numpy(X_train_1_np).float()
X_val_1 = th.from_numpy(X_val_1_np).float()
y_train_1 = th.from_numpy(y_train_1_np).float()
y_val_1 = th.from_numpy(y_val_1_np).float()

In [6]:
#data = th.FloatTensor(np.array([5, 15, 25, 35, 45, 55]).reshape(-1, 1))

X_train_1 = X_train_1.tag("DO2 training X data")
X_train_1 = X_train_1.describe("number of samples and features")

data_ptr = X_train_1.send(duet, searchable=True)

`searchable` is deprecated please use `pointable` in future


In [7]:
y_train_1 = y_train_1.tag("DO2 training y data")
y_train_1 = y_train_1.describe("number of samples and features")

target_ptr = y_train_1.send(duet, searchable=True)

`searchable` is deprecated please use `pointable` in future


In [8]:
train_num_1 = sy.lib.python.Int(X_train_1.shape[0])
train_num_1.tag("size of DO2's training data")
train_num_1.describe("size of DO2's training data")
train_num_1.send(duet, searchable=True)

`searchable` is deprecated please use `pointable` in future


In [9]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: d293b7fa16894bb1a27aed991d952170>,[DO2 training X data],number of samples and features,<class 'torch.Tensor'>
1,<UID: 8dc4d2073b4f419990e61f4ff41d5190>,[DO2 training y data],number of samples and features,<class 'torch.Tensor'>
2,<UID: fc735c2487f048a0a1524d28eaed05a9>,[size of DO2's training data],size of DO2's training data,<class 'syft.lib.python.Int'>


In [10]:
duet.requests.add_handler(
    action="accept",
    print_local=True,  # print the result in your notebook
)